In [14]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [16]:
os.getenv("LANGCHAIN_TRACING_V2")
os.getenv("LANGCHAIN_API_KEY")

'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'

In [17]:
def get_model():
    from langchain_community.chat_models.zhipuai import ChatZhipuAI
    from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage, AIMessage
    model = ChatZhipuAI(model='glm-4',
                        # temperature 需要精确到小数后两位，太多的话接口会报错
                        temperature=0.7,
                        api_key='00d2ffc1109bb54764d67a63c7c3826c.CWRsU68TB9pbmeig', max_tokens=1024)
    return model

In [46]:
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage,AIMessage

model = get_model()
messages = [
    SystemMessage(content="你是一个翻译官，帮我将下面的信息翻译为意大利语。并且只返回翻译的内容"),
    # AIMessage(content="好的"),
    HumanMessage(content="翻译：你好。我是一个学生"),
]

In [47]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
chain = model | parser
chain.invoke(messages)

'Il tuo messaggio tradotto in italiano è:\n\n"Ciao. Sono uno studente."'

In [48]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "你是一个翻译官，帮我将下面的信息翻译为 {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
result = prompt_template.invoke({"language": "意大利语", "text": "你好。我是一个学生"})

result

ChatPromptValue(messages=[SystemMessage(content='你是一个翻译官，帮我将下面的信息翻译为 意大利语:'), HumanMessage(content='你好。我是一个学生')])

In [49]:
result.to_messages()

[SystemMessage(content='你是一个翻译官，帮我将下面的信息翻译为 意大利语:'),
 HumanMessage(content='你好。我是一个学生')]

In [50]:
chain = prompt_template | model | parser

In [51]:
chain.invoke({"language": "意大利语", "text": "我是一个学生"})

'Certamente, per favore fornisci il testo che desideri tradurre in italiano.'

In [52]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好")])

AIMessage(content='你好👋！我是人工智能助手智谱清言，可以叫我小智🤖，很高兴见到你，欢迎问我任何问题。', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 6, 'total_tokens': 38}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-7c3a9524-64e6-4c2f-ad93-f5d21d1336dd-0')

In [54]:
model.invoke(
    [
        HumanMessage(content="你好， 我是bob"),
        AIMessage(content="你好"),
        HumanMessage(content="我叫什么"),
    ]
)

AIMessage(content='你告诉我你叫Bob。如果有其他问题或需要帮助，请随时告诉我。', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 15, 'total_tokens': 34}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-6b10f7f2-bf0a-4b9c-aca3-90652f882662-0')

In [55]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [56]:
config = {"configurable": {"session_id": "abc2"}}

In [57]:
response = with_message_history.invoke(
    [HumanMessage(content="你好，我是bob")],
    config=config,
)

response.content

Parent run a92af55a-0af8-4b24-90cb-deec596808be not found for run bb8ff789-d8f9-4508-b23b-22288624e07f. Treating as a root run.


'你好，Bob！有什么我可以帮助你的吗？'

In [60]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

Parent run 7ba4d7b1-4d21-41cd-8b1d-cdc1f1b85fce not found for run 9aeb52e1-e41f-45ea-adb1-a86b9d1908ce. Treating as a root run.


'Your name is Bob, as you stated when we began this conversation. If you have any other questions or need assistance, feel free to ask!'

In [62]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run da40e6c0-cbc0-48f1-8bc8-f957f7b66380 not found for run e2af0d4c-b641-4d1d-b5f9-b77e01cb2441. Treating as a root run.


"You stated that your name is Bob at the beginning of our conversation. Is there anything else you'd like to know or discuss, Bob?"

In [63]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [64]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello, Bob! How can I assist you today? If you have any questions or need information on a topic, feel free to ask.'